In [93]:
from datetime import datetime, timedelta
from dotenv import load_dotenv
from ipywidgets import (
    interact, interactive, fixed, interact_manual, Layout
)
import ipywidgets as widgets
import json
import os
import time
import requests

load_dotenv()

TOKEN = os.getenv('TOKEN')

HEADERS = {"Authorization": f"Bearer {TOKEN}"}

GRAPHQL_URL = "https://api.github.com/graphql"

In [94]:
start_date = widgets.DatePicker(
    value=datetime.now() - timedelta(days=365),
    description='Start Date',
    disabled=False
)
end_date = widgets.DatePicker(
    value=datetime.now() - timedelta(days=1),
    description='End Date',
    disabled=False
)

button = widgets.Button(
    description="Click me",
    disabled=False,
    button_style="", # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click me",
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

def fdate(start, end):
    print(
        "\nstart date: {}, end date: {}\n".format(start, end)
    )

date_out = widgets.interactive_output(
    fdate, {
        "start": start_date,
        "end": end_date
    }
)

display(start_date, end_date, button, date_out)

DatePicker(value=datetime.datetime(2024, 7, 18, 13, 12, 56, 545461), description='Start Date', step=1)

DatePicker(value=datetime.datetime(2025, 7, 17, 13, 12, 56, 548085), description='End Date', step=1)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

In [106]:
def get_unique_commit_authors_between_dates(since, until):
    """
    > Endpoint #1
    
    - Accepts: A start and end date.
    
    - Returns: A list of unique commit authors within the date constraints supplied.
    """
    print(f"\nSearching for commit authors since {since} until {until}\n")
    q = (
        "{ rateLimit { limit cost remaining resetAt } "
        " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
        " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
        f" history( first: 100 since: \"{since}\" until: \"{until}\") "
        " { pageInfo { hasNextPage startCursor endCursor } "
        " edges { node { author { name email }}}}}}}}}"
    )
    try:
        authors = set()
        while q is not None:
            r = requests.post(
                GRAPHQL_URL,
                json={"query": q},
                headers=HEADERS
            )
            if r.status_code == 429:
                time.sleep(60)
            if r.status_code != 200:
                print("uh oh")
                raise
            response = r.json()
            # print(response)
            commits = response["data"]["repository"]["ref"]["target"]["history"]["edges"]
            print(f"Found {len(commits)} commits since {since}")
            for commit in commits:
                authors.add(commit["node"]["author"]["email"])
            if response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["hasNextPage"] == True:
                after = response["data"]["repository"]["ref"]["target"]["history"]["pageInfo"]["endCursor"]
                q = (
                    "{ rateLimit { limit cost remaining resetAt } "
                    " repository(name: \"OpenRA\", owner: \"OpenRA\") { "
                    " ref(qualifiedName: \"bleed\") { target { ... on Commit { "
                    f" history( first: 100 after {after}) "
                    " { pageInfo { hasNextPage startCursor endCursor } "
                    " edges { node { author { name email }}}}}}}}}"
                )
            else:
                q = None
        print(f"\n***Unique authors: \n\n{authors}\n\n")
    except Exception as e:
        print(e)
        raise

get_unique_commit_authors_between_dates(
    f"{start_date.value}T00:00:00",
    f"{end_date.value}T00:00:00"
)


Searching for commit authors since 2025-07-01T00:00:00 until 2025-07-18T00:00:00

Found 15 commits since 2025-07-01T00:00:00

***Unique authors: 

{'68965911+toasted-dev@users.noreply.github.com', '41052878+darkademic@users.noreply.github.com', '119738087+michaeldgg2@users.noreply.github.com', 'matthias@mailaender.name', 'jms.happycat@gmail.com', 'ashleynewson@smartsim.org.uk', 'anvilvapre@users.noreply.github.com'}


